In [96]:
import numpy as np
import random
import pandas as pd
import plotly_express as px
import plotly.graph_objects as go

In [32]:
class GenAlg:

    def __init__(self, population_size, k_crossover, mutation_prob, rates, sigma):
        self.population_size = population_size
        self.k = k_crossover
        self.mutation_prob = mutation_prob
        self.number_of_stocks = len(rates)

        self.rates = rates.copy()
        self.sigma = sigma.copy()

        self.random_pop()
    


    def random_pop(self):
        self.population = np.random.rand(self.population_size,self.number_of_stocks)
        for  i in range(self.population.shape[0]):
            self.population[i,] = self.normalize(self.population[i,])
    
    def normalize(self,vector):
        sum_of_vector = sum(vector)
        for i in range(len(vector)):
            vector[i] =  vector[i]/sum_of_vector
        return vector
    
    def evaluate(self):
        best_fitness = None
        best = None
        for vector in self.population:
            f = self.fitness(vector)
            if best is None or f > best_fitness:
                best = vector
                best_fitness = f
        return best_fitness,best

    def fitness(self,vector):
        rate = np.sum(vector * self.rates)
        volatility = np.sqrt(np.dot(vector.T,np.dot(self.sigma,vector)))

        return rate/volatility

    def get_lucky_one(self):
        random_num = random.randrange(len(self.population))
        return self.population[random_num,:]
    
    def select_parents(self,lucky_one):
        # if lucky_one exists then remove him
        if lucky_one is not None:
            pop = np.ones(shape=(self.population_size-1,self.number_of_stocks))
            i = 0
            for vec in self.population:
                if not (vec == lucky_one).all():
                    pop[i,:] = vec
                    i+=1
        else:
            pop = self.population.copy()
        
        new_pop = np.array(sorted(pop,key=self.fitness,reverse=True))

        return new_pop[:len(new_pop)//2,:]

    def create_children(self,parents):
        
        children = np.zeros(shape=(self.population_size//2,self.number_of_stocks))
        np.random.shuffle(parents)
        first_p, second_p = map(list,np.split(parents,2))
        for i in range(len(parents)//2):
            parent1 = first_p.pop(0)
            parent2 = second_p.pop(0)

            child1, child2 = self.k_crossover(parent1,parent2)

            child1 = self.mutation(child1)
            child2 = self.mutation(child2)

            child1 = self.normalize(child1)
            child2 = self.normalize(child2)

            children[i,:] = child1
            children[i+len(parents)//2,:] = child2

        return children 
    
    def k_crossover(self,x,y):

        slices = set()
        while len(slices) < self.k-1:
            slice_index = random.choice([i for i in range(1,len(x)) if i not in slices])
            if slice_index not in slices:
                slices.add(slice_index)

        slices = sorted(list(slices))
        slices.append(self.number_of_stocks)

        new_x = np.zeros(self.number_of_stocks)
        new_y = np.zeros(self.number_of_stocks)

        first = 0
        switch = True
        for second in slices:
            if switch:
                new_x[first:second] = x[first:second]
                new_y[first:second] = y[first:second]
            else:
                new_x[first:second] = y[first:second]
                new_y[first:second] = x[first:second]
            first = second
            switch = not switch

        return new_x,new_y

    def mutation(self,child):
        for i in range(len(child)):
            if random.random() < self.mutation_prob:
                child[i] += random.gauss(0.25,0.1)
        return child

    def solve(self,max_generations, goal):
        
        best_fit, best = self.evaluate()
        
        for iteration in range(max_generations):
            #print(iteration,'iteration',end=' ')
            if best_fit > goal:
                break

            lucky_one = None
            if len(self.population)%2 == 1:
                lucky_one = self.get_lucky_one()

            parents = self.select_parents(lucky_one)

            children = self.create_children(parents)

            if lucky_one is not None:
                lucky_one = lucky_one.reshape((1,len(lucky_one)))
                self.population = np.concatenate((lucky_one,parents,children),axis=0)
            else:
                self.population = np.concatenate((parents,children),axis=0)

            best_fit, best = self.evaluate()
            #print('best ratio:',best_fit)
        #print('finnished')
        return best,best_fit

In [3]:
wanted_stocks = ['GOOG', 'SPG', 'GOOGL', 'MSFT', 'GD', 'ACN', 'COP', 'F', 'BAC', 'GS',
                'NVDA', 'AIG', 'MS', 'WFC', 'ORCL', 'XOM', 'TGT', 'LOW', 'EXC', 'COST',
                'AXP', 'BK', 'JPM', 'COF', 'CSCO', 'DHR', 'UNH', 'CVS', 'LLY', 'CVX',
                'MET', 'AMT', 'CRM', 'BLK', 'RTX', 'MCD', 'TMO', 'LIN', 'ADBE', 'EMR',
                'USB', 'UPS', 'TSLA', 'PFE', 'PM']
stocks = pd.read_csv('data/sap100_data_08112021.csv',index_col=0).loc[:,wanted_stocks]
returns = stocks/stocks.shift(1)-1
rates = returns.mean() * 252
sigma = returns.cov() * 252
g = GenAlg(population_size=100, k_crossover=5, mutation_prob=0.01, rates=rates,sigma=sigma)

best,best_fit = g.solve(10,8)
print(best,best_fit)
# print(g.population[0,:],'\n',g.population[1,:])

# g.k_crossover(g.population[0,:],g.population[1,:])

# test_pop = np.zeros(shape=(5,45))
# for i in range(5):
#     test_pop[i,i] = 1
# print(test_pop)
# for i in range(5):
#     print(g.fitness(test_pop[i,:]))
# g.population = test_pop
# v = g.get_lucky_one()
# print('lucky one: ',v)
# print('paretns')
# print(g.select_parents(v))

0 iteration best ratio: 3.9814872156712546
1 iteration best ratio: 4.012415536157719
2 iteration best ratio: 4.026275876124628
3 iteration best ratio: 4.026275876124628
4 iteration best ratio: 4.083732457995888
5 iteration best ratio: 4.257872511485389
6 iteration best ratio: 4.274747397460453
7 iteration best ratio: 4.347876055794951
8 iteration best ratio: 4.347876055794951
9 iteration best ratio: 4.347876055794951
finnished
[ 0.05010221  0.01634367  0.01254702  0.0331552   0.03374839  0.02453896
  0.00628114  0.04471909  0.03680227  0.03178333  0.04570675  0.02309245
  0.01078585  0.01244529  0.04592989  0.02417768  0.01741712  0.03957004
  0.00410536  0.0216843   0.01537904  0.04732551  0.02497932  0.02394333
  0.0026971   0.03252971  0.01060076  0.04529616  0.05205395  0.01097592
 -0.11360534  0.04550091  0.0453456   0.0031177   0.02666841  0.01721195
  0.03262874  0.01462731  0.0052458   0.00169118  0.01247602  0.02549887
  0.01513958  0.03503559  0.03270087] 4.347876055794951


In [4]:
a = [0.01114155, 0.02112081, 0.0026389 , 0.04158689 ,0.02911615 ,0.00810653,
 0.00953936, 0.01291289, 0.03460458, 0.0172124,  0.02812051, 0.00615008,
 0.03882499, 0.01603352 ,0.03222546, 0.02002055, 0.01096212, 0.00879385,
 0.02504149, 0.02297195, 0.04096784, 0.04078883, 0.00404298 ,0.00336519,
 0.02037814, 0.0395612 , 0.041175,   0.03722034, 0.01283837, 0.02809674,
 0.0232359 , 0.03603028 ,0.04134022, 0.00582411, 0.01888116, 0.03858949,
 0.00809903, 0.03437509, 0.03463708 ,0.00438613 ,0.02915961, 0.01188739,
 0.00890677, 0.02284441, 0.01624411]
max(a)

0.04158689

In [5]:
b = [0.00022345, 0.04564275, 0.04497704, 0.04376311, 0.0033903 , 0.03373989,
 0.04167013,0.04474843, 0.03991494, 0.013077 ,  0.04262159 ,0.01209769,
 0.03196004, 0.01024975, 0.03971374 ,0.01291337, 0.00605673, 0.03494775,
 0.01216985, 0.00211064, 0.00081961, 0.03155706, 0.01703879, 0.02700199,
 0.00440683 ,0.01209849, 0.02187265, 0.03169752, 0.02678447 ,0.00618878,
 0.0080071 , 0.03951205 ,0.01762882, 0.01950138, 0.00031214, 0.02950566,
 0.04465236, 0.01634321 ,0.01039963, 0.00432588, 0.02807762, 0.00894588,
 0.03424738, 0.0192949,  0.02379159]
max(b)

0.04564275

In [6]:
def print_stats(vec):
    w = np.array(vec)
    r = np.sum(w*rates)
    print('rate',r)
    v = np.sqrt(np.dot(w.T,np.dot(sigma,w)))
    print('volatility:',v)
    print('shapre ratio',r/v)

In [7]:
print_stats(a)
print()
print_stats(b)

rate 0.4701581146929166
volatility: 0.12546335975815912
shapre ratio 3.747373859580875

rate 0.542997231753761
volatility: 0.1344691863204556
shapre ratio 4.038079255270691


In [22]:
import statistics

In [177]:
m_tri = []
m_jeden = []
for p in range(10, 25, 3):
    tri, jeden  = [], []
    for _ in range(100):
        g = GenAlg(population_size=100, k_crossover=5, mutation_prob=0.01, rates=rates,sigma=sigma)
        best,best_fit = g.solve(p,8)
        menej_ako_jedno_percenta = [i for i in best if i < 0.01]
        menej_ako_tri_percenta = [i for i in best if i < 0.03]
        tri.append(len(menej_ako_tri_percenta))
        jeden.append(len(menej_ako_jedno_percenta))
    m_tri.append(tri)
    m_jeden.append(jeden)
print([i for i in range(10,25,3)])
print(m_tri)
print(m_jeden)
'''
print(best,best_fit)
print('menej ako 1%', len(menej_ako_jedno_percenta))
print('menej ako 3%', len(menej_ako_tri_percenta))
'''


[10, 13, 16, 19, 22]
[[35, 37, 39, 38, 33, 32, 36, 38, 30, 28, 40, 38, 34, 32, 32, 35, 37, 32, 41, 38, 39, 37, 37, 41, 39, 32, 39, 37, 33, 34, 40, 38, 35, 39, 30, 31, 39, 35, 36, 40, 33, 37, 32, 38, 39, 33, 32, 35, 30, 36, 30, 35, 38, 34, 38, 27, 39, 36, 38, 37, 34, 40, 36, 41, 33, 41, 37, 39, 38, 40, 32, 36, 41, 38, 32, 38, 34, 41, 37, 36, 40, 40, 39, 36, 31, 32, 34, 35, 33, 34, 37, 40, 40, 38, 35, 41, 31, 32, 36, 38], [38, 39, 33, 37, 37, 36, 36, 39, 39, 31, 37, 39, 31, 37, 34, 35, 38, 37, 39, 33, 37, 33, 30, 39, 35, 41, 38, 36, 38, 39, 40, 39, 38, 33, 35, 37, 40, 39, 40, 29, 37, 33, 38, 28, 37, 40, 36, 37, 39, 36, 36, 39, 39, 38, 36, 38, 35, 37, 34, 39, 36, 38, 37, 36, 34, 40, 34, 36, 34, 34, 30, 35, 36, 30, 36, 36, 33, 38, 40, 37, 37, 34, 38, 35, 37, 35, 31, 35, 36, 40, 35, 32, 37, 38, 40, 41, 35, 39, 33, 34], [38, 35, 37, 37, 37, 34, 28, 36, 36, 38, 39, 38, 38, 35, 34, 38, 33, 34, 39, 36, 38, 33, 39, 40, 35, 34, 36, 39, 38, 33, 36, 37, 35, 34, 32, 36, 34, 32, 30, 35, 38, 34, 33, 3

"\nprint(best,best_fit)\nprint('menej ako 1%', len(menej_ako_jedno_percenta))\nprint('menej ako 3%', len(menej_ako_tri_percenta))\n"

In [178]:
df_menej_jeden = pd.DataFrame(m_jeden)
df_menej_jeden['index'] = [i for i in range(10,25,3)]
df_menej_jeden  = df_menej_jeden.set_index('index')
df_menej_jeden = df_menej_jeden.reset_index()
df_menej_jeden

,index,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,10,18,15,16,13,18,13,10,14,11,...,20,16,21,19,18,21,17,18,16,11
1,13,19,18,14,12,17,15,14,20,18,...,11,15,16,14,15,19,13,13,10,16
2,16,21,15,21,14,19,19,18,16,17,...,14,18,12,19,23,13,18,24,22,13
3,19,19,18,23,16,18,15,19,19,15,...,19,22,22,19,15,19,13,15,19,17
4,22,19,21,22,20,22,17,29,17,16,...,20,22,16,23,20,17,15,22,18,16


In [179]:
fig = px.histogram(df_menej_jeden, color = 'index', opacity=0.5, barmode='overlay')
fig.update_traces(xbins_size=1)
fig.show()

In [180]:
the_best = []
menej = []
for k in range(2,25):
    best_fit_list = []
    menej_jedno = []
    for _ in range(50):
        g = GenAlg(population_size=100, k_crossover=k, mutation_prob=0.01, rates=rates,sigma=sigma)
        best,best_fit = g.solve(10,8)
        best_fit_list.append(best_fit)
        menej_ako_jedno_percenta = [i for i in best if i < 0.01]
        menej_jedno.append(len(menej_ako_jedno_percenta))
    the_best.append(statistics.mean(best_fit_list))
    menej.append(statistics.mean(menej_jedno))

print(the_best)
print(menej)


KeyboardInterrupt: 

In [ ]:
viac = [int(45-i) for i in menej]
df_k = pd.DataFrame({"k":[i for i in range(2,25)],"ratio": the_best, "> 1%":viac})
df_k

,k,ratio,> 1%
0,2,4.209633,43
1,3,4.207521,43
2,4,4.233026,43
3,5,4.237508,43
4,6,4.232400,43
5,7,4.242764,43
6,8,4.258676,43
7,9,4.250918,43
8,10,4.238205,43
9,11,4.244309,43


In [ ]:
fig = go.Figure()
#mutation prob = 0.1, pop size 100
fig.add_trace(go.Scatter(x=df_k['k'], y=df_k['ratio'], name="ratio", yaxis="y1"))
fig.add_trace(go.Scatter(x=df_k['k'], y=df_k['> 1%'], name="> 1%", yaxis="y2"))
fig.update_layout(
    xaxis=dict(title='k'),
    yaxis=dict(
        title="ratio",
        titlefont=dict(
            color="#1f77b4"
        ),
        tickfont=dict(
            color="#1f77b4"
        )
    ),
    yaxis2=dict(
        title="> 1%",
        titlefont=dict(
            color="red"
        ),
        tickfont=dict(
            color="red"
        ),
        anchor="free",
        overlaying="y",
        side="right",
        position=1
    ))
fig.show()

In [ ]:
the_best = []
menej = []
for p in [0.005, 0.010, 0.015, 0.020, 0.025, 0.030, 0.035, 0.040, 0.045, 0.050]:
    best_fit_list = []
    menej_jedno = []
    for _ in range(100):
        g = GenAlg(population_size=100, k_crossover=8, mutation_prob=p, rates=rates,sigma=sigma)
        best,best_fit = g.solve(10,8)
        best_fit_list.append(best_fit)
        menej_ako_jedno_percenta = [i for i in best if i < 0.01]
        menej_jedno.append(len(menej_ako_jedno_percenta))
    the_best.append(statistics.mean(best_fit_list))
    menej.append(statistics.mean(menej_jedno))

print(the_best)
print(menej)

[4.253402391404574, 4.240468755728645, 4.244981346974212, 4.236227869169, 4.237263834299535, 4.238344474144795, 4.226049139489052, 4.221051678145077, 4.2218181264829004, 4.227376505191429]
[1.84, 1.6, 1.9, 2.09, 2.38, 1.83, 2.39, 2.83, 3.15, 3.3]


In [ ]:
viac = [int(45-i) for i in menej]
df_p = pd.DataFrame({"p":[0.005, 0.010, 0.015, 0.020, 0.025, 0.030, 0.035, 0.040, 0.045, 0.050],"ratio": the_best, "> 1%":viac})
df_p

,p,ratio,> 1%
0,0.005,4.253402,43
1,0.010,4.240469,43
2,0.015,4.244981,43
3,0.020,4.236228,42
4,0.025,4.237264,42
5,0.030,4.238344,43
6,0.035,4.226049,42
7,0.040,4.221052,42
8,0.045,4.221818,41
9,0.050,4.227377,41


In [ ]:
fig = go.Figure()
#k = 8, pop size 100
fig.add_trace(go.Scatter(x=df_p['p'], y=df_p['ratio'], name="ratio", yaxis="y1"))
fig.add_trace(go.Scatter(x=df_p['p'], y=df_p['> 1%'], name="> 1%", yaxis="y2"))
fig.update_layout(
    xaxis=dict(title='p'),
    yaxis=dict(
        title="ratio",
        titlefont=dict(
            color="#1f77b4"
        ),
        tickfont=dict(
            color="#1f77b4"
        )
    ),
    yaxis2=dict(
        title="> 1%",
        titlefont=dict(
            color="red"
        ),
        tickfont=dict(
            color="red"
        ),
        anchor="free",
        overlaying="y",
        side="right",
        position=1
    ))
fig.show()

In [ ]:
g = GenAlg(population_size=100, k_crossover=8, mutation_prob=0.01, rates=rates,sigma=sigma)

best,best_fit = g.solve(10,8)
print(best,best_fit)

[0.03845245 0.00087674 0.03496243 0.02453384 0.00711184 0.03044989
 0.00143412 0.10335219 0.00203175 0.01902237 0.04109037 0.03976671
 0.00100643 0.00339921 0.15215102 0.00298171 0.00875201 0.03468123
 0.0109422  0.00418775 0.00616967 0.00326929 0.01239119 0.01409109
 0.00228115 0.01087108 0.03132437 0.03722236 0.02566128 0.0005705
 0.00476897 0.0740307  0.03228559 0.00252054 0.00207925 0.04259115
 0.03511779 0.01664765 0.02536945 0.00670688 0.01183224 0.00071782
 0.01645228 0.01502736 0.0088141 ] 4.23576802043203
